In [2]:
from djitellopy import Tello
import cv2
import numpy as np
import time

def detection_center(det):
    """Center x, y normalized to [-0.5, 0.5]"""
    cx = (det[3] + det[5]) / 2.0 - 0.5
    cy = (det[4] + det[6]) / 2.0 - 0.5
    return (cx, cy)

def norm(vec):
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    best_det = None
    min_dist = float('inf')
    for det in detections[0, 0]:
        if det[2] > 0.4 and int(det[1]) == 1:  # 'person'
            center = detection_center(det)
            distance = norm(center)
            if distance < min_dist:
                min_dist = distance
                best_det = det
    return best_det

# Load COCO class names
with open('COCO/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')

COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

model = cv2.dnn.readNet(model='COCO/frozen_inference_graph.pb',
                        config='COCO/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt',
                        framework='TensorFlow')

tello = Tello()
print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False
kill_switch_engaged = False  # Kill switch flag

# Control parameters
area_target = 0.25
area_tolerance = 0.05
move_step_cm = 30
pixel_threshold = 40  # pixels for left/right decision

last_move_time = time.time()
move_interval = 0.8  # seconds between moves

try:
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Object detection
        blob = cv2.dnn.blobFromImage(frame, size=(300, 300), mean=(104, 117, 123), swapRB=True)
        model.setInput(blob)
        detections = model.forward()
        det = closest_detection(detections)

        if det is not None:
            class_id = int(det[1])
            class_name = class_names[class_id - 1]
            color = COLORS[class_id]

            x1 = int(det[3] * w)
            y1 = int(det[4] * h)
            x2 = int(det[5] * w)
            y2 = int(det[6] * h)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            label = f"{class_name} {det[2]:.2f}"
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            box_area = (x2 - x1) * (y2 - y1)
            image_area = w * h
            area_ratio = box_area / image_area
            area_error = area_ratio - area_target

            bbox_center_x = (x1 + x2) / 2
            frame_center_x = w / 2
            x_offset = bbox_center_x - frame_center_x

            if drone_in_air:
                if kill_switch_engaged:
                    # Continuous max forward velocity
                    print(f"[KILL SWITCH ENGAGED] Full send forward")
                    tello.send_rc_control(0, 100, 0, 0)  # (lr, fb, ud, yaw)
                else:
                    # Stop continuous motion
                    tello.send_rc_control(0, 0, 0, 0)

                    if (time.time() - last_move_time) > move_interval:
                        moved = False

                        if area_error < -area_tolerance:
                            print(f"Moving forward {move_step_cm} cm")
                            tello.move_forward(move_step_cm)
                            moved = True
                        elif area_error > area_tolerance:
                            print(f"Moving back {move_step_cm} cm")
                            tello.move_back(move_step_cm)
                            moved = True

                        if abs(x_offset) > pixel_threshold:
                            if x_offset > 0:
                                print(f"Moving right {move_step_cm} cm")
                                tello.move_right(move_step_cm)
                            else:
                                print(f"Moving left {move_step_cm} cm")
                                tello.move_left(move_step_cm)
                            moved = True

                        if not moved:
                            print("Within tolerance. Hovering.")

                        last_move_time = time.time()

        cv2.imshow("Tello Tracking Feed", frame)

        key = cv2.waitKey(1) & 0xFF

        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne.")
            tello.move_up(50)

        if drone_in_air:
            if key == ord('w'):
                tello.move_forward(50)
            elif key == ord('a'):
                tello.move_left(50)
            elif key == ord('s'):
                tello.move_back(50)
            elif key == ord('d'):
                tello.move_right(50)

        if key == ord('k'):
            kill_switch_engaged = not kill_switch_engaged
            state = "ENGAGED" if kill_switch_engaged else "DISENGAGED"
            print(f"Kill switch {state}.")
            if not kill_switch_engaged:
                tello.send_rc_control(0, 0, 0, 0)  # stop movement immediately

        if key == ord('q'):
            if drone_in_air:
                tello.land()
                print("Drone has landed.")
            break

except Exception as e:
    print(f"Error: {e}")
    if drone_in_air:
        tello.land()

except KeyboardInterrupt:
    print("Interrupted. Landing drone.")
    if drone_in_air:
        tello.land()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'


Connecting to Tello...
Battery level: 61%


[INFO] tello.py - 438 - Send command: 'streamon'
[INFO] tello.py - 462 - Response streamon: 'ok'


Press 't' to takeoff, 'q' to land and quit.


2025-04-19 20:40:38.978 Python[21549:4281531] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
[INFO] tello.py - 438 - Send command: 'takeoff'
[INFO] tello.py - 462 - Response takeoff: 'ok'
[INFO] tello.py - 438 - Send command: 'up 50'


...

[INFO] tello.py - 471 - Send command (no response expected): 'rc 0 100 0 0'


[KILL SWITCH ENGAGED] Full send forward


[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'error'
[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'error'
[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'error'
[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'error'
[INFO] tello.py - 438 - Send command: 'land'


Error: Command 'land' was unsuccessful for 4 tries. Latest response:	'error'


[INFO] tello.py - 462 - Response land: 'error'
[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'ok'
[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'error'
[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'ok'
